# Случайные леса
__Суммарное количество баллов: 10__

В этом задании вам предстоит реализовать ансамбль деревьев решений, известный как случайный лес, применить его к публичным данным и сравнить его эффективность с ансамблями из самых популярных библиотек.

In [30]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas
import random
import matplotlib.pyplot as plt
import matplotlib
import copy

import math

In [31]:
def gini(x):
    labels_cnt = dict()
    for label in x:
        if label in labels_cnt.keys():
            labels_cnt[label] += 1
        else:
            labels_cnt[label] = 1
    gini_coeff = 0
    for label in labels_cnt.keys():
        p_label = labels_cnt[label] / len(x)
        gini_coeff += p_label * (1 - p_label)

    return gini_coeff
    
def entropy(x):
    labels_cnt = dict()
    for label in x:
        if label in labels_cnt.keys():
            labels_cnt[label] += 1
        else:
            labels_cnt[label] = 1
    entropy_coeff = 0
    for label in labels_cnt.keys():
        p_label = labels_cnt[label] / len(x)
        entropy_coeff -= p_label * math.log(p_label, 2)
    return entropy_coeff

def gain(left_y, right_y, criterion):
    size_left = len(left_y)
    size_right = len(right_y)
    size_all = size_left + size_right
    
    Q = criterion(left_y + right_y) - (size_right/size_all) * criterion(right_y) - (size_left/size_all) * criterion(left_y)
    SplitInfo = 0
    
    labels_cnt = dict()
    for label in (left_y + right_y):
        if label in labels_cnt.keys():
            labels_cnt[label] += 1
        else:
            labels_cnt[label] = 1
    for label in labels_cnt.keys():
        p_label = labels_cnt[label] / size_all
        SplitInfo -= p_label * math.log(p_label, 2)
    
    return Q / SplitInfo

In [32]:
class DecisionTreeLeaf:
    def __init__(self, y = None):
        self.y = y

class DecisionTreeNode:
    def __init__(self, split_dim, split_value, left, right):
        self.split_dim = split_dim
        self.split_value = split_value
        self.left = left
        self.right = right
        

In [33]:
def make_split(X, y, min_size, critetion):
    n = len(X)
    m = len(X[0])
    max_gain = 0
    split_dim = -1
    split_value = 0
    
    for dim in range(m):
        for value in X[:, dim]:
            left_y, right_y = [], []
            for i in range(len(X)):
                if X[i][dim] < value:
                    left_y.append(y[i])
                else:
                    right_y.append(y[i])
            
            if len(left_y) >= min_size and len(right_y) >= min_size:  
                cur_gain = gain(left_y, right_y, critetion)
                if cur_gain > max_gain:
                    max_gain = cur_gain
                    split_dim = dim
                    split_value = value
                
    return split_dim, split_value

In [34]:
def make_node(tree, X, y, depth):
    new_node = None
        
    if len(y) == 0: # no data in the group
        return None
    if np.all(y == y[0]): # every label is the in the group
        return DecisionTreeLeaf(y[0])
    if depth < 0: # depth limit reached
        return None

    if tree.criterion == "gini":
        split_dim, split_value = make_split(X, y, tree.min_samples_leaf, gini)
    else:
        split_dim, split_value = make_split(X, y, tree.min_samples_leaf, entropy)
    if split_dim == -1:
        return DecisionTreeLeaf(np.bincount(y).argmax())

    X_left = X[X[:, split_dim] < split_value]
    X_right = X[X[:, split_dim] >= split_value]
    y_left = y[X[:, split_dim] < split_value]
    y_right = y[X[:, split_dim] >= split_value]

    return DecisionTreeNode(split_dim, split_value
                            , make_node(tree, X_left, y_left, depth - 1), make_node(tree, X_right, y_right, depth - 1))

### Задание 1 (3 балла)
Реализуем сам Random Forest. Идея очень простая: строим `n` деревьев, а затем берем модальное предсказание. Используйте реализацию дерева из HW3.

#### Параметры конструктора
`n_estimators` - количество используемых для предсказания деревьев.

Остальное - параметры деревьев.

#### Методы
`fit(X, y)` - строит `n_estimators` деревьев по выборке `X`.

`predict(X)` - для каждого элемента выборки `X` возвращает самый частый класс, который предсказывают для него деревья.

In [55]:
class RandomForestClassifier:
    def __init__(self, criterion="gini", max_depth=None, min_samples_leaf=1, max_features="auto", n_estimators=10):
        self.criterion = criterion
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features
        self.n_estimators = n_estimators
    
    def fit(self, X, y):
        self.roots = []
        
        all_indexes = list(range(1, len(y)))
        for i in range(self.n_estimators):
            cur_indexes = random.sample(all_indexes, len(y) // 2)
            cur_X = X[cur_indexes, :]
            cur_y = y[cur_indexes]
            
            self.roots.append(make_node(self, cur_X, cur_y, self.max_depth))

    
    def predict(self, X):
        answer = []
        for elem in X:
            cur_predictions = []
            for i in range(self.n_estimators):
                cur_node = self.roots[i]
                while type(cur_node) == DecisionTreeNode:
                    if elem[cur_node.split_dim] < cur_node.split_value:
                        cur_node = cur_node.left
                    else:
                        cur_node = cur_node.right
                cur_predictions.append(cur_node.y)
            answer.append(np.bincount(cur_predictions).argmax())
        return answer

In [56]:
X = [[1, 1], 
     [2, 2],
     [1, 7], 
     
     [2, 7], 
     [8, 6],
     [5, 5]
    ]
y = [0, 0, 0, 1, 1, 1]
X = np.array(X)
y = np.array(y)

In [57]:
forest = RandomForestClassifier("gini", max_depth=5, min_samples_leaf=1, max_features="auto", n_estimators=3)
forest.fit(X, y)
forest.predict(X) ## работает!

[0, 0, 0, 1, 1, 1]

### Задание 3 (2 балла)
Оптимизируйте по `AUC` на кроссвалидации (размер валидационной выборки - 20%) параметры своей реализации `Random Forest`: 

максимальную глубину деревьев из [2, 3, 5, 7, 10], количество деревьев из [5, 10, 20, 30, 50, 100]. 

Постройте `ROC` кривую (и выведите `AUC` и `accuracy`) для лучшего варианта.

Подсказка: можно построить сразу 100 деревьев глубины 10, а потом убирать деревья и
глубину.

In [ ]:
# YOUR_CODE

### Задание 4 (3 балла)
Часто хочется понимать, насколько большую роль играет тот или иной признак для предсказания класса объекта. Есть различные способы посчитать его важность. Один из простых способов сделать это для Random Forest выглядит так:
1. Посчитать out-of-bag ошибку предсказания `err_oob` (https://en.wikipedia.org/wiki/Out-of-bag_error)
2. Перемешать значения признака `j` у объектов выборки (у каждого из объектов изменится значение признака `j` на какой-то другой)
3. Посчитать out-of-bag ошибку (`err_oob_j`) еще раз.
4. Оценкой важности признака `j` для одного дерева будет разность `err_oob_j - err_oob`, важность для всего леса считается как среднее значение важности по деревьям.

Реализуйте функцию `feature_importance`, которая принимает на вход Random Forest и возвращает массив, в котором содержится важность для каждого признака.

In [ ]:
def feature_importance(rfc):
    raise NotImplementedError()

def most_important_features(importance, names, k=20):
    # Выводит названия k самых важных признаков
    idicies = np.argsort(importance)[::-1][:k]
    return np.array(names)[idicies]

Протестируйте решение на простом синтетическом наборе данных. В результате должна получиться точность `1.0`, наибольшее значение важности должно быть у признака с индексом `4`, признаки с индексами `2` и `3`  должны быть одинаково важны, а остальные признаки - не важны совсем.

In [ ]:
def synthetic_dataset(size):
    X = [(np.random.randint(0, 2), np.random.randint(0, 2), i % 6 == 3, 
          i % 6 == 0, i % 3 == 2, np.random.randint(0, 2)) for i in range(size)]
    y = [i % 3 for i in range(size)]
    return np.array(X), np.array(y)

X, y = synthetic_dataset(1000)
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X, y)
print("Accuracy:", np.mean(rfc.predict(X) == y))
print("Importance:", feature_importance(rfc))

Проверьте, какие признаки важны для датасетов cancer и spam?

_Ваш ответ_

### Задание 5 (2 балла)
В качестве альтернативы попробуем библиотечные реализации ансамблей моделей. 

1. [CatBoost](https://catboost.ai/docs/)
2. [XGBoost](https://xgboost.readthedocs.io/en/latest/)
3. [LightGBM](https://lightgbm.readthedocs.io/en/latest/)


Установите необходимые библиотеки. 
Возможно, потребуется установка дополнительных пакетов

In [ ]:
!pip install lightgbm
!pip install catboost
!pip install xgboost

Также, как и реализованный нами RandomForest, примените модели для наших датасетов.

Для стандартного набора параметров у каждой модели нарисуйте `ROC` кривую и выведите `AUC` и `accuracy`.

Посчитайте время обучения каждой модели (можно использовать [timeit magic](https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-timeit)). 

Сравните метрики качества и скорость обучения моделей. Какие выводы можно сделать?

In [1]:
# YOUR_CODE

_Ваш ответ_